In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 247 kB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 40.9 MB/s 
     |████████████████████████████████| 298 kB 46.2 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 3.3 MB 33.2 MB/s 
     |████████████████████████████████| 9.1 MB 22.8 MB/s 
     |████████████████████████████████| 1.7 MB 38.7 MB/s 
     |████████████████████████████████| 3.3 MB 34.5 MB/s 
     |████████████████████████████████| 61 kB 510 kB/s 
     |████████████████████████████████| 596 kB 38.6 MB/s 
     |████████████████████████████████| 895 kB 22.0 MB/s 
     |████████████████████████████████| 140 kB 48.7 MB/s 
     |████████████████████████████████| 97 kB 4.7 MB/s 
     |████████████████████████████████| 180 kB 46.0 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 243 kB 43.4 MB/s 
     |████████████████████████████████| 132 kB 32.5 MB/s 
     |█████████████████

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)  

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/TA_project/Text-Analytics-Project

/content/drive/MyDrive/TA_project/Text-Analytics-Project


In [ ]:
from transformers import logging

logging.set_verbosity_warning()

In [ ]:
import pandas as pd

df = pd.read_csv('movie_dataset_classification.csv',index_col=0)
df.dropna(subset=['Plot'], inplace=True)
df.drop_duplicates(subset=['Title','Plot'], inplace=True)
#removing low frequency genres
counts = df.Genre.value_counts()
counts = list(counts[counts < 150].index)
df = df[~df['Genre'].isin(counts)]

In [ ]:
df

,Release Year,Title,Director,Cast,Plot,Genre
0,1903.0,The Great Train Robbery,Edwin S. Porter,NaN,The film opens with two bandits breaking into ...,western
1,1904.0,The Suburbanite,Wallace McCutcheon,NaN,The film is about a family who move to the sub...,comedy
2,1906.0,From Leadville to Aspen: A Hold-Up in the Rockies,Francis J. Marion and Wallace McCutcheon,NaN,The film features a train traveling through th...,action
3,1907.0,Daniel Boone,Wallace McCutcheon and Ediwin S. Porter,"William Craven, Florence Lawrence",Boone's daughter befriends an Indian maiden as...,historical
4,1907.0,How Brown Saw the Baseball Game,Unknown,Unknown,Before heading out to a baseball game at a nea...,comedy
...,...,...,...,...,...,...
40117,NaN,Ready Player One,NaN,[],Wade Watts is a teenager who lives in a slum w...,sport
40118,2011.0,From the Rough,Pierre Bagley,"['Taraji P. Henson', 'Tom Felton', 'Michael Cl...",Catana Starks is the coach for the women's swi...,sport
40119,NaN,Brink!,Greg Beeman,"['Erik von Detten', 'Christina Vidal', 'Robin ...","Andy ""Brink"" Brinker and his in-line skating c...",sport
40120,2003.0,Open Water (film),Chris Kentis,"['Blanchard Ryan', 'Daniel Travis']",Daniel Kintner and Susan Watkins are frustrate...,sport


In [ ]:
from sklearn.model_selection import train_test_split

X = df['Plot'].values
y = df['Genre'].values

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, shuffle=True)

In [ ]:
(train_inputs, validation_inputs,
 train_labels, validation_labels) = train_test_split(X_train, y_train,
                                                     random_state=42,
                                                     test_size=0.20,
                                                     stratify=y_train)
train_df = pd.DataFrame({'text':train_inputs,'label':train_labels})
validation_df = pd.DataFrame({'text':validation_inputs,'label':validation_labels})
train_df,validation_df

(                                                    text  label
 0      A rivalry between U.S. cavalry captains result...     18
 1      Lawyer Harry (Robert Young) and his wife Tacey...      4
 2      Three elementary school boys — Brad (Adam LaVo...      4
 3      Mourners are gathered around an old man at his...     14
 4      The film starts with an old man, Harry Banks (...      7
 ...                                                  ...    ...
 19619  Yusuf Ağa, the landlord of the tiny southeast ...      4
 19620  God (George Burns) appears as a kindly old man...      4
 19621  A teenaged Hans Christian Andersen daydreams i...      2
 19622  Abraham Schwartz, a bookkeeper living in a sma...      6
 19623  Street-smart white detective Danny Bassett (Al...     12
 
 [19624 rows x 2 columns],
                                                    text  label
 0     Sam Carter (Patrick) is a jeweler's clerk who ...     16
 1     A once loyal soldier faces many challenges aft...      6

In [ ]:
# Optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 4
model_args.overwrite_output_dir = True

In [ ]:
# Create a ClassificationModel
model = ClassificationModel("bert", "bert-base-uncased",args=model_args,num_labels=len(set(y_train)))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
model.train_model(train_df,eval_df=validation_df,model_args=model_args)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/19624 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/2453 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/2453 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/2453 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/2453 [00:00<?, ?it/s]

(9812, 1.167628872627829)

In [ ]:
test_df = pd.DataFrame({'text':X_test,'label':y_test})

In [ ]:
results = model.evaluate(test_df,'test')

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/11215 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1402 [00:00<?, ?it/s]

KeyError: ignored

In [ ]:
import numpy as np

y_pred = np.argmax(results[1],axis=1)
y_pred

array([ 1, 14, 14, ..., 16,  4, 18])

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)

0.4729380294248774